<H1>  Install and Import

In [0]:
!pip install PyDrive
!pip install JSAnimation

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials

import os
import cv2
from PIL import Image as Im
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras
import h5py
import random
from tensorflow.keras.layers import SpatialDropout2D, Conv2D, MaxPooling2D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, LeakyReLU, Add, SpatialDropout3D,Conv3D, MaxPooling3D, Activation, BatchNormalization, PReLU,concatenate,Layer, InputSpec, Flatten,Dense, Softmax, Dropout
from tensorflow.keras import Model,initializers, regularizers, constraints,Sequential
from tensorflow.keras.optimizers import Adam, Adamax, RMSprop, SGD

from JSAnimation import IPython_display

<h1> Tunneling to TensorBoard

In [0]:
import os
import requests
import shutil
import subprocess
import tensorflow as tf

__all__ = [
  'install_ngrok', 
  'launch_tensorboard',
]

def __shell__(cmd, split=True):
  # get_ipython().system_raw(cmd)
  result = get_ipython().getoutput(cmd, split=split)
  if result and not split:
    result = result.strip('\n')
  return result

def install_ngrok(bin_dir="/tmp"):
  TARGET_DIR = bin_dir
  CWD = os.getcwd()
  is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
  if is_grok_avail:
    print("ngrok installed")
  else:
    import platform
    plat = platform.platform()
    if 'x86_64' in plat:
      
      os.chdir('/tmp')
      print("calling wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip ..." )
      get_ipython().system_raw( "wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip" )
      print("calling unzip ngrok-stable-linux-amd64.zip ...")
      get_ipython().system_raw( "unzip ngrok-stable-linux-amd64.zip" )
      os.rename("ngrok", "{}/ngrok".format(TARGET_DIR))
      os.remove("ngrok-stable-linux-amd64.zip")
      is_grok_avail = os.path.isfile(os.path.join(TARGET_DIR,'ngrok'))
      os.chdir(TARGET_DIR)
      if is_grok_avail:
        print("ngrok installed. path={}".format(os.path.join(TARGET_DIR,'ngrok')))
      else:
        raise ValueError( "ERROR: ngrok not found, path=".format(TARGET_DIR) )
    else:
      raise NotImplementedError( "ERROR, ngrok install not configured for this platform, platform={}".format(plat))
    os.chdir(CWD)
    return
    
def launch_tensorboard(bin_dir="/tmp", log_dir="/tmp", retval=False):
  install_ngrok(bin_dir)
    
  if not tf.gfile.Exists(log_dir):  tf.gfile.MakeDirs(log_dir)
  ps = __shell__("ps -ax")
  is_tensorboard_running = len([f for f in ps if "tensorboard" in f ]) > 0
  is_ngrok_running = len([f for f in ps if "ngrok" in f ]) > 0
  print("status: tensorboard={}, ngrok={}".format(is_tensorboard_running, is_ngrok_running))

  if not is_tensorboard_running:
    get_ipython().system_raw(
        'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
        .format(log_dir)
    )
    is_tensorboard_running = True
    
  if not is_ngrok_running:
    get_ipython().system_raw('{}/ngrok http 6006 &'.format(bin_dir))
    is_ngrok_running = True
  import time
  time.sleep(3)
  retval = requests.get('http://localhost:4040/api/tunnels')
  tensorboard_url = retval.json()['tunnels'][0]['public_url'].strip()
  print("tensorboard url=", tensorboard_url)
  if retval:
    return tensorboard_url

In [0]:
# set paths
ROOT = %pwd
LOG_DIR = os.path.join(ROOT, 'log1')

# will install `ngrok`, if necessary
# will create `log_dir` if path does not exist
launch_tensorboard( bin_dir=ROOT, log_dir=LOG_DIR )

<h1> Get Data from Drive

In [0]:
## connect to drive and Locally download the file

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1HLYgLqxFrrWy1Q58knC85jX0PGgLgYeo'})
download.GetContentFile('Assignment')

In [0]:
!unzip ./Assignment -d ./Assignment2

<h1> Pre-Preocessing and Data Augmentation

<h2>Dict having file names</h2>frames[i] contains all the images in lec i <br>and<br> labels[i] has corresponding labels

In [0]:
frames_dir = next(os.walk('./Assignment2/frames/'))[1]
frames_dir.sort()
frames = {}
labels = {}
for i in range(len(frames_dir)):
  a=next(os.walk('./Assignment2/frames/' + frames_dir[i] + '/'))[2]
  a.sort()
  frames[i] = ['./Assignment2/frames/' + frames_dir[i] + '/' + s for s in a]
  labels[i] = np.genfromtxt('./Assignment2/labels/' + frames_dir[i] +'.csv', delimiter=',')    

<h2>Img --> np array</h2>return size= 240, 320, 3

In [0]:
def im(image_address):
  img = cv2.imread(image_address)
  res = cv2.resize(img, dsize=(320, 240), interpolation=cv2.INTER_CUBIC)
  return res


Total number of initial data points

In [0]:
s=0
for i in labels:
  s=s+np.sum(labels[i])
print(s)
s1=0
for i in range(35):
  s1=s1+labels[i].shape[0]
print(s1)
plt.bar(["zero","one"],[29137,1130])
plt.ylabel("Class count in data")


<h2>Data Augmentation</h2>

In [0]:
indexes = {}
for i in range(len(frames_dir)):
  indexes[i] = []
  for j in range(len(labels[i])):
    if labels[i][j]==1 and j>1 and j<len(labels[i])-2:
      indexes[i].append(j)

r={}
for i in range(len(frames_dir)):
  
  r[i] = [[indexes[i][0]-1,indexes[i][0],indexes[i][0]+1]]
  r[i].append([indexes[i][0],indexes[i][0]+1,indexes[i][0]+2])

  for j in range(1,int(np.sum(labels[i]))-2):
    
    r[i].append([indexes[i][j]-1,indexes[i][j],indexes[i][j]+1])
    r[i].append([indexes[i][j],indexes[i][j]+1,indexes[i][j]+2])
    
    for k in range(1, int(np.sum(labels[i]))-2):
      if j!=k:
        r[i].append([indexes[i][j]-1,indexes[i][j],indexes[i][k]+1])
        r[i].append([indexes[i][j],indexes[i][k]+1,indexes[i][k]+2])
        
a0=[]
for i in range(35):
      a = [random.randint(1,int(labels[i].shape[0]-3)) for k in range(int(len(r[i])*0.21))]
      for j in a:
        if labels[i][j]==0 and labels[i][j-1]==0:
          a0.append([i,j])

Number of data points after augmentation

In [0]:
s=0
for i in range(35):
  s=s+len(r[i])
print("Label one: ", s)

Label one:  98200


In [0]:
q=[]
for i in range(35):
  p1=0
  p2=0
  for j in range(len(labels[i])):
    if labels[i][j] == 1:
      p1=p2
      p2=j
      q.append(p2-p1)

a = plt.hist(q,230)
plt.ylabel("Number of occurences in dataset")
plt.xlabel("Duration between imp frames")
plt.show()

<h2>Normalize and save in h5py file

In [0]:
with h5py.File('./X.h5','w') as hdf:
  z=0
  dset = hdf.create_dataset("X", (25000,240 ,320 ,9))
  for i in range(35):
      a = [random.randint(0,len(r[i])-1) for k in range(int(len(r[i])*0.102))]
      for j in a:
        dset[z] = np.concatenate([im(frames[i][r[i][j][0]]),im(frames[i][r[i][j][1]]),im(frames[i][r[i][j][2]])], axis=2)/255
        z=z+1
        if z%1000==0:
          print(z)
      true_upto = z-1
  p=-1
  while z<25000:
      p=p+1
      dset[z] = np.concatenate([im(frames[a0[p][0]][a0[p][1]-1]),im(frames[a0[p][0]][a0[p][1]]),im(frames[a0[p][0]][a0[p][1]+1])], axis=2)/255
      z=z+1
      if z%1000==0:
           print(z)

In [0]:
print(true_upto)

10001


<h1>
3D_CNN 

<h2>Data Generator

In [0]:
class DataGenerator(tensorflow.keras.utils.Sequence):

    def __init__(self, list_IDs, labels_partition, batch_size=2, shuffle = True):

        self.batch_size = batch_size
        self.labels = labels_partition
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
          np.random.shuffle(self.indexes)


    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs)/self.batch_size))
      
    def __getitem__(self, index):
      #'Generate one batch of data'
      indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
      
      list_IDs_temp = [self.list_IDs[k] for k in indexes]
      list_IDs_temp.sort()

      with h5py.File('./X.h5','r') as hdf:
        X=np.array(hdf['X'][list_IDs_temp,:,:,:])
      y = np.zeros((self.batch_size))
      for p in range(self.batch_size):
          y[p] = self.labels[list_IDs_temp[p]]

      
      return (np.array([[X[0,:,:,:3],X[0,:,:,3:6],X[0,:,:,6:]],[X[1,:,:,:3],X[1,:,:,3:6],X[1,:,:,6:]],[X[2,:,:,:3],X[2,:,:,3:6],X[2,:,:,6:]],[X[3,:,:,:3],X[3,:,:,3:6],X[3,:,:,6:]]]),y)


In [0]:
# Datasets
xxx=list(range(25000))
random.shuffle(xxx)
partition = {"train" : xxx[:24000], "validation": xxx[24000:]}
labels_partition = {}
for i in xxx:
  if i <10001:
    labels_partition[i] = 1
  else:
    labels_partition[i] = 0

# Parameters
params = {'batch_size': 4,
          'shuffle': True}

s0=0
s1=0
training_generator = DataGenerator(partition['train'], labels_partition, **params)
import time
start = time.time()
for i in range(10):
  X0,y0=training_generator[i]
  s0=s0+np.sum(y0[:,0])
  s1=s1+np.sum(y0[:,1])
print(s0,s1)
end = time.time()
print(end-start)

In [0]:
print(len(training_generator))

In [0]:
print(X0.shape)

<h2>Model</h2>

In [0]:
def Block(input_layer,
           n_filters,
           batch_normalization=True,
           kernel=(3, 3, 3),
           strides=(1,3,3),
           dropout_rate=0.4,
           data_format="channels_first"):

    layer = Conv3D(n_filters, kernel, padding='same', strides=strides)(input_layer)
    if batch_normalization:
        layer = BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    layer = MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),padding='SAME')(layer)
    dropout = SpatialDropout3D(rate=dropout_rate, data_format=data_format)(layer)
    return dropout

In [0]:
def MODEL1(input_shape=(360,450,9),
                      n_filter=3,
                      depth=3, 
                      dropout_rate=0.3,
                      optimizer=Adam, 
                      initial_learning_rate=5e-4,
                      loss_function=tensorflow.keras.losses.binary_crossentropy):
  graph0 = tf.Graph()
  with graph0.as_default():
    inputs = Input(input_shape)
    layer1 = BatchNormalization()(inputs)
    for level_number in range(depth):
      layer1 = Block(layer1,n_filters=n_filter[level_number])
    layer1 = Flatten()(layer1)
    dense_1 = Dense(512, activation = 'relu')(layer1)
    dense_2 = Dense(10 , activation = 'relu')(dense_1)
    activation_block = Dense(1 , activation = 'sigmoid')(dense_2)
    model = Model(inputs=inputs, outputs=activation_block)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function,metrics = ['accuracy'])
    writer = tf.summary.FileWriter(logdir='log1', graph=graph0)
    writer.flush()
    return model

In [0]:

model = MODEL1(input_shape=(3,240,320,3), n_filter=[32,64,128], depth=2, dropout_rate=0.42856,
                      optimizer=Adam, initial_learning_rate=0.0001,
                      loss_function=tensorflow.keras.losses.binary_crossentropy)

model.summary()

<h3>Training

In [0]:
def log_dir_name(initial_learning_rate=0.0001,depth=4,n_filters=5,dropout_rate=0.4,optimizer='Adam',loss_function='dice'):

    # The dir-name for the TensorBoard log-dir.
    s = "./log1/lr_{0:.0e}_d_{1}_f_{2}_{3}_o_{4}_l_{5}/"

    log_dir = s.format(initial_learning_rate,
                       depth,
                       n_filters,
                       dropout_rate,
                       optimizer,
                       loss_function)

    return log_dir

In [0]:
# Datasets
xxx=list(range(15000))
random.shuffle(xxx)
partition = {"train" : xxx[:14900], "validation": xxx[14900:]}
labels_partition = {}
for i in xxx:
  if i <10001:
    labels_partition[i] = 1
  else:
    labels_partition[i] = 0

# Parameters
params = {'batch_size': 4,
          'shuffle': True}

# Generators
training_generator = DataGenerator(partition['train'], labels_partition, **params)
validation_generator = DataGenerator(partition['validation'], labels_partition, **params)                                                                                                            

cb_1=tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=0, mode='auto')
cb_2=tensorflow.keras.callbacks.ModelCheckpoint(filepath='./weights2.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

log_dir = log_dir_name()

cb_3 = tensorflow.keras.callbacks.TensorBoard(
        log_dir=log_dir,
        histogram_freq=0,
        write_graph=True,
        write_grads=False,
        write_images=False)

# results = model.fit(x = ,y = ,batch_size=batch_size, epochs=5, verbose=1, callbacks=[cb_1,cb_2,cb_3], validation_split=0.02, shuffle=True)

results = model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                   epochs=50,
                   callbacks=[cb_1,cb_2,cb_3])

<h2>Testing MODEL</h2>

In [0]:
print(model.predict(validation_generator[2]))
print(validation_generator[2][1])

In [0]:
validation_generator = DataGenerator(partition['validation'], labels_partition, **params)
prediction = model.predict_generator(generator=validation_generator)

In [0]:
print(prediction[:10])

<h1>A time-distributed ConvNet and passing the features to an RNN, in one network

<h2>Soft Label Assignment

In [0]:
soft_labels = {}
for i in range(35):
  previous = -10
  soft = np.zeros(labels[i].shape[0])
  for j in reversed(range(labels[i].shape[0])):
    if (labels[i][j]==1):
      previous = j
    soft[j] = np.exp(-1 * np.square(j - previous)/(2*2))
  soft_labels[i] = soft

In [0]:
c=soft_labels[0][:100]
c=np.flip(c,axis=0)
plt.plot(range(100),c)
plt.ylabel("Label value")
plt.xlabel("frame no.")

<h2>Data Generator</h2>

In [0]:
class DataGenerator1(tensorflow.keras.utils.Sequence):
  
    def __init__(self, list_IDs, labels_partition, batch_size=2, shuffle = True):

        self.batch_size = batch_size
        self.labels = labels_partition
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
          np.random.shuffle(self.indexes)

    def __len__(self):
        #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs)/self.batch_size))
      
    def __getitem__(self, index):
      #'Generate one batch of data'
      indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
      
      list_IDs_temp = [self.list_IDs[k] for k in indexes]
      list_IDs_temp.sort()

      with h5py.File('./X.h5','r') as hdf:
        X=np.array(hdf['X'][list_IDs_temp,:,:,:])
      y = np.zeros((self.batch_size,2))
      for p in range(self.batch_size):
          y[p][0] = self.labels[list_IDs_temp[p]]
      y[:,1] = 1-y[:,0]

<h2>Model</h2>

In [0]:
def lrcn(input_shape = (16,240,320,3),
         optimizer = Adam,
         loss_function = tensorflow.keras.losses.binary_crossentropy, 
         initial_learning_rate = 0.0001):  
#     graph = tf.Graph()
#     with graph.as_default():
        model = Sequential()

        model.add(TimeDistributed(Conv2D(64, (7, 7), strides=(2, 2),activation='relu', padding='same'), input_shape=input_shape))
        model.add(TimeDistributed(Conv2D(64, (3,3), kernel_initializer="he_normal", activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(256, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
        
        model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(Conv2D(512, (3,3), padding='same', activation='relu')))
        model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))

        model.add(TimeDistributed(Flatten()))

        model.add(Dropout(0.5))
        model.add(Bidirectional(LSTM(256, return_sequences=True, dropout=0.5)))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function,metrics = ['accuracy'])
#     writer = tf.summary.FileWriter(logdir='log1', graph=graph)
#     writer.flush()    
        return model

In [0]:
model = lrcn()
model.summary()

<h2>Model Training

In [0]:
X_train = []
Y_train = []
for i in range(1):
  x_train=[]
  y_train=[]
  for t in tqdm(range(4)):
    for j in range(int(len(frames[i])//16)):
      x=[]
      y=[]
      for k in range(16):
        image = im(frames[i*4+t][j*16+k])
        x.append((image - image.mean()) / image.std())
        y.append(soft_labels[i*4+t][j*16+k])
      x_train.append(np.array(x))
      y_train.append(np.array(y))
  X_train.append(np.array(x_train))
  Y_train.append(np.array(y_train))
model.fit(X_train,Y_train,batch_size=4,epochs=5)

In [0]:
print(len(x_train))
print(x_train[0].shape)

4150
(240, 320, 3)


<h1> Model 3
Extracting features from each frame with a ConvNet(inception net) and passing the sequence to a separate RNN

In [0]:
def lstm(input_shape = (None,4096),
         initial_learning_rate=0.0001,
         optimizer=Adam,
        loss_function=tensorflow.keras.losses.binary_crossentropy):
        model = Sequential()
        model.add(LSTM(2048, return_sequences=True,
                       input_shape=input_shape,
                       dropout=0.5))
        model.add(Dense(512, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function,metrics = ['accuracy'])

        return model

In [0]:
model = lstm()
model.summary()

<h1>2D cnn with 3 stacked frames.

In [0]:
def Block(input_layer,
           n_filters,
           batch_normalization=True,
           kernel=(3, 3),
           dropout_rate=0.4,
           data_format="channels_first"):

    layer = Conv2D(n_filters, kernel, padding='same')(input_layer)
    if batch_normalization:
        layer = BatchNormalization()(layer)
    layer = Activation('relu')(layer)
    layer = MaxPooling2D(pool_size=(2, 2), strides=(2, 2),padding='SAME')(layer)
    dropout = SpatialDropout2D(rate=dropout_rate, data_format=data_format)(layer)
    return dropout

In [0]:
def MODEL1(input_shape=(360,450,9),
                      n_filter=3,
                      depth=3, 
                      dropout_rate=0.3,
                      optimizer=Adam, 
                      initial_learning_rate=5e-4,
                      loss_function=tensorflow.keras.losses.binary_crossentropy):
  graph0 = tf.Graph()
  with graph0.as_default():
    inputs = Input(input_shape)
    layer1 = BatchNormalization()(inputs)
    for level_number in range(depth):
      layer1 = Block(layer1,n_filters=n_filter[level_number])
    layer1 = Flatten()(layer1)
    dense_1 = Dense(512, activation = 'relu')(layer1)
    dense_2 = Dense(10 , activation = 'relu')(dense_1)
    activation_block = Dense(1 , activation = 'sigmoid')(dense_2)
    model = Model(inputs=inputs, outputs=activation_block)
    model.compile(optimizer=optimizer(lr=initial_learning_rate), loss=loss_function,metrics = ['accuracy'])
  writer = tf.summary.FileWriter(logdir='log1', graph=graph0)
  writer.flush()
  return model

In [0]:
model = MODEL1(input_shape=(240,320,9), n_filter=[32,64,128,256,125], depth=5, dropout_rate=0.42856,
                      optimizer=Adam, initial_learning_rate=0.0001,
                      loss_function=tensorflow.keras.losses.binary_crossentropy)

model.summary()